# Recommender Notebook
---

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

%matplotlib inline

In [2]:
ratings = pd.read_csv('./data/veggie_ratings.csv')
ratings.head()

,Timestamp,Artichokes,Arugula,Asparagus,Bush beans (green beans),Pole beans(green beans),Beets,Bok Choy,Broccoli,Brussel Sprouts,...,Radishes,Rhubarb,Rutabaga,Shallots,Spinach,Summer Squash,Winter Squash,Swiss Chard,Tomatoes,Turnips
0,10/12/2021 17:47:21,5,5,5,4,4,1,3,5,5,...,1,2,2,4,5,1,1,3,5,2
1,10/12/2021 17:57:50,2,4,2,4,2,1,1,3,1,...,1,1,1,3,5,1,1,1,4,1
2,10/12/2021 18:17:26,3,3,5,3,3,1,1,4,3,...,1,2,1,5,5,1,1,1,4,1
3,10/12/2021 18:18:15,4,2,1,2,2,2,3,3,3,...,3,2,2,2,2,2,2,2,1,2
4,10/12/2021 18:21:26,3,2,5,5,5,1,1,5,1,...,1,1,1,1,1,1,1,1,5,1


In [3]:
ratings['user_id'] = ratings.index
ratings.drop(columns='Timestamp', inplace=True)

In [4]:
veg = ratings.columns[0:-1]
veg

Index(['Artichokes', 'Arugula', 'Asparagus', 'Bush beans (green beans)',
       'Pole beans(green beans)', 'Beets', 'Bok Choy', 'Broccoli',
       'Brussel Sprouts', 'Cabbage', 'Carrots', 'Cauliflower', 'Celery',
       'Chives', 'Collards', 'Corn', 'Cucumbers', 'Eggplant', 'Garlic', 'Kale',
       'Leeks', 'Lettuce', 'Mustard Greens', 'Onions', 'Parsnips', 'Peas',
       'Peppers', 'Potatoes', 'Pumpkin', 'Radishes', 'Rhubarb', 'Rutabaga',
       'Shallots', 'Spinach', 'Summer Squash', 'Winter Squash', 'Swiss Chard',
       'Tomatoes', 'Turnips'],
      dtype='object')

In [5]:
veg_dict = {k: v for v, k in enumerate(veg)}
veg_dict

{'Artichokes': 0,
 'Arugula': 1,
 'Asparagus': 2,
 'Bush beans (green beans)': 3,
 'Pole beans(green beans)': 4,
 'Beets': 5,
 'Bok Choy': 6,
 'Broccoli': 7,
 'Brussel Sprouts': 8,
 'Cabbage': 9,
 'Carrots': 10,
 'Cauliflower': 11,
 'Celery': 12,
 'Chives': 13,
 'Collards': 14,
 'Corn': 15,
 'Cucumbers': 16,
 'Eggplant': 17,
 'Garlic': 18,
 'Kale': 19,
 'Leeks': 20,
 'Lettuce': 21,
 'Mustard Greens': 22,
 'Onions': 23,
 'Parsnips': 24,
 'Peas': 25,
 'Peppers': 26,
 'Potatoes': 27,
 'Pumpkin': 28,
 'Radishes': 29,
 'Rhubarb': 30,
 'Rutabaga': 31,
 'Shallots': 32,
 'Spinach': 33,
 'Summer Squash': 34,
 'Winter Squash': 35,
 'Swiss Chard': 36,
 'Tomatoes': 37,
 'Turnips': 38}

In [6]:
ratings = ratings.melt(id_vars = 'user_id')

In [7]:
ratings.shape

(4875, 3)

In [8]:
ratings.head()

,user_id,variable,value
0,0,Artichokes,5
1,1,Artichokes,2
2,2,Artichokes,3
3,3,Artichokes,4
4,4,Artichokes,3


In [9]:
ratings['veggie_id'] = ratings['variable'].map(veg_dict)

In [10]:
ratings.head()

,user_id,variable,value,veggie_id
0,0,Artichokes,5,0
1,1,Artichokes,2,0
2,2,Artichokes,3,0
3,3,Artichokes,4,0
4,4,Artichokes,3,0


In [11]:
ratings = ratings.rename(columns={'variable': 'veggie_name', 'value': 'rating'})

In [12]:
ratings.head()

,user_id,veggie_name,rating,veggie_id
0,0,Artichokes,5,0
1,1,Artichokes,2,0
2,2,Artichokes,3,0
3,3,Artichokes,4,0
4,4,Artichokes,3,0


In [13]:
pivot = pd.pivot_table(ratings, index='veggie_name', columns='user_id', values='rating')
pivot.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
veggie_name,,,,,,,,,,,,,,,,,,,,,
Artichokes,5,2,3,4,3,4,4,4,5,2,...,1,4,2,5,3,5,5,1,4,3
Arugula,5,4,3,2,2,1,2,5,5,3,...,1,5,1,1,2,5,2,4,2,4
Asparagus,5,2,5,1,5,4,4,5,2,5,...,1,5,4,5,5,5,5,4,3,5
Beets,1,1,1,2,1,4,1,4,5,1,...,1,5,1,1,1,2,2,1,4,2
Bok Choy,3,1,1,3,1,4,4,4,1,5,...,1,4,1,5,3,5,1,1,3,3


In [14]:
sparse_pivot = sparse.csr_matrix(pivot)

In [15]:
dists = cosine_distances(sparse_pivot)
dists

array([[0.        , 0.10090689, 0.0960003 , ..., 0.11025035, 0.13839955,
        0.12054969],
       [0.10090689, 0.        , 0.06951891, ..., 0.08271811, 0.14015046,
        0.11921901],
       [0.0960003 , 0.06951891, 0.        , ..., 0.06335072, 0.14632511,
        0.09233123],
       ...,
       [0.11025035, 0.08271811, 0.06335072, ..., 0.        , 0.14420635,
        0.09115027],
       [0.13839955, 0.14015046, 0.14632511, ..., 0.14420635, 0.        ,
        0.11312013],
       [0.12054969, 0.11921901, 0.09233123, ..., 0.09115027, 0.11312013,
        0.        ]])

In [16]:
similarities = cosine_similarity(sparse_pivot)

In [17]:
similarities

array([[1.        , 0.89909311, 0.9039997 , ..., 0.88974965, 0.86160045,
        0.87945031],
       [0.89909311, 1.        , 0.93048109, ..., 0.91728189, 0.85984954,
        0.88078099],
       [0.9039997 , 0.93048109, 1.        , ..., 0.93664928, 0.85367489,
        0.90766877],
       ...,
       [0.88974965, 0.91728189, 0.93664928, ..., 1.        , 0.85579365,
        0.90884973],
       [0.86160045, 0.85984954, 0.85367489, ..., 0.85579365, 1.        ,
        0.88687987],
       [0.87945031, 0.88078099, 0.90766877, ..., 0.90884973, 0.88687987,
        1.        ]])

In [18]:
recommender_df = pd.DataFrame(dists, columns=pivot.index, index=pivot.index)
recommender_df.head()

veggie_name,Artichokes,Arugula,Asparagus,Beets,Bok Choy,Broccoli,Brussel Sprouts,Bush beans (green beans),Cabbage,Carrots,...,Radishes,Rhubarb,Rutabaga,Shallots,Spinach,Summer Squash,Swiss Chard,Tomatoes,Turnips,Winter Squash
veggie_name,,,,,,,,,,,,,,,,,,,,,
Artichokes,0.000000,0.100907,0.096000,0.155125,0.113214,0.099324,0.111638,0.121389,0.115719,0.113122,...,0.123694,0.156485,0.123201,0.097834,0.096716,0.116625,0.112679,0.110250,0.138400,0.120550
Arugula,0.100907,0.000000,0.069519,0.144015,0.134040,0.079348,0.085539,0.094629,0.104827,0.091337,...,0.113072,0.126614,0.137019,0.078349,0.065999,0.107274,0.120956,0.082718,0.140150,0.119219
Asparagus,0.096000,0.069519,0.000000,0.162596,0.110035,0.047394,0.073614,0.059927,0.079939,0.060300,...,0.119069,0.122251,0.118955,0.083974,0.042015,0.077114,0.127524,0.063351,0.146325,0.092331
Beets,0.155125,0.144015,0.162596,0.000000,0.150132,0.174875,0.148601,0.157619,0.152078,0.175084,...,0.112925,0.170194,0.135040,0.140444,0.156092,0.185146,0.144281,0.161970,0.131624,0.181067
Bok Choy,0.113214,0.134040,0.110035,0.150132,0.000000,0.098938,0.118112,0.123444,0.112459,0.120789,...,0.128226,0.135218,0.088200,0.094901,0.109990,0.105838,0.104958,0.133730,0.123619,0.096847


In [40]:
recommender_df.to_csv('./data/recommender_df.csv')

In [44]:
recommender_df['Beets'].sort_values()[1:11]

veggie_name
Radishes           0.112925
Turnips            0.131624
Rutabaga           0.135040
Shallots           0.140444
Arugula            0.144015
Swiss Chard        0.144281
Brussel Sprouts    0.148601
Onions             0.149240
Bok Choy           0.150132
Leeks              0.151530
Name: Beets, dtype: float64

In [21]:
def recommend_veggie(search_term):
    titles = pivot.filter(like = search_term, axis='index').index
    
    for title in titles:
        print(title)
        print('Average rating:', pivot.loc[title].mean())
        print('Number of ratings:', pivot.loc[title].count())
        print()
        print('10 most similar veggies:')
        print(recommender_df[title].sort_values()[1:11])
        print('*' * 37)
        print()

In [22]:
recommend_veggie('Beets')

Beets
Average rating: 2.36
Number of ratings: 125

10 most similar veggies:
veggie_name
Radishes           0.112925
Turnips            0.131624
Rutabaga           0.135040
Shallots           0.140444
Arugula            0.144015
Swiss Chard        0.144281
Brussel Sprouts    0.148601
Onions             0.149240
Bok Choy           0.150132
Leeks              0.151530
Name: Beets, dtype: float64
*************************************



Read in our veg_info dataframe.

In [23]:
veg_info = pd.read_csv('./data/veg_info.csv')
veg_info.head()

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use
0,Artichokes,full sunlight,well drained,high,"6, 7, 8, 9",perennial,3,3 to 6 feet,2 to 4 feet,85 - 100 days,violet,light green,spreading mass/upright,high,Not native to North America,none,edible landscaping
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers
2,Asparagus,full sunlight or partial shade,well drained,low,"2, 3, 4, 5, 6, 7, 8",perennial,2,5 to 9 feet,2 to 2.5 feet,2 - 3 years,green,light green,upright,low,Not native to North America,"Bears ornamental fruit - small, bright red ber...",edible landscaping
3,Bush beans,full,well drained,medium,"3, 4, 5, 6, 7, 8, 9, 10",annual,1,1 to 3 feet,1 to 2 feet,50 - 60 days,white,medium green,spreading mass,low,Not native to North America but cultivated wor...,Bears ornamental fruit - some varieties have p...,edible landscaping
4,Pole beans,full,well drained,medium,"3, 4, 5, 6, 7, 8, 9, 10",annual,1,5 to 10 feet,1 to 2 feet,60 - 80 days,white,medium green,climbing/vine,low,Not native to North America but cultivated wor...,none,edible landscaping


Check dtypes:

1. hardiness_zones - need to be changed to a list of integers
2. flower_color - we'll have to be prepared to handle the NaNs

In [24]:
# def str_to_lst(x):
#     lst = x
#     lst = lst.split(",")
#     lst = [int(i) for i in lst]
#     return lst

In [25]:
# veg_info['hardiness_zones'] = veg_info['hardiness_zones'].apply(str_to_lst)

In [26]:
veg_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   plant            39 non-null     object
 1   sunlight         39 non-null     object
 2   soil_condition   39 non-null     object
 3   moisture_level   39 non-null     object
 4   hardiness_zones  39 non-null     object
 5   lifecycle        39 non-null     object
 6   difficulty       39 non-null     int64 
 7   height           39 non-null     object
 8   spread           39 non-null     object
 9   time_to_mature   39 non-null     object
 10  flower_color     19 non-null     object
 11  foliage_color    39 non-null     object
 12  shape            39 non-null     object
 13  frost_tolerance  39 non-null     object
 14  special_att1     39 non-null     object
 15  special_att2     39 non-null     object
 16  special_use      39 non-null     object
dtypes: int64(1), object(16)
memory usage:

In [45]:
veg_lst = recommender_df[['Beets']].sort_values(by='Beets')[1:11]
veg_lst

veggie_name,Beets
veggie_name,
Radishes,0.112925
Turnips,0.131624
Rutabaga,0.135040
Shallots,0.140444
Arugula,0.144015
Swiss Chard,0.144281
Brussel Sprouts,0.148601
Onions,0.149240
Bok Choy,0.150132


Create a list of recommended vegetables to parse our veg_info table for

In [28]:
veg_lst = veg_lst.index.tolist()
veg_lst

['Radishes',
 'Turnips',
 'Rutabaga',
 'Shallots',
 'Arugula',
 'Swiss Chard',
 'Brussel Sprouts',
 'Onions',
 'Bok Choy',
 'Leeks']

Iterate through veg_info and print out the rows which match our veggies from the veg_list

In [29]:
for num in range(0, veg_info.shape[0]):
    for veg in veg_lst:
            if veg_info['plant'][num] == veg:
                print(veg_info.loc[[num]])

     plant                        sunlight soil_condition moisture_level  \
1  Arugula  full sunlight or partial shade  low fertility         medium   

               hardiness_zones lifecycle  difficulty       height  \
1  3, 4, 5, 6, 7, 8, 9, 10, 11    annual           1  1 to 2 feet   

          spread time_to_mature flower_color foliage_color  \
1  0.5 to 1 foot    4 - 7 weeks        white  medium green   

                     shape frost_tolerance                 special_att1  \
1  cushion, mound, upright          medium  Not native to North America   

  special_att2     special_use  
1         none  edible flowers  
      plant                        sunlight soil_condition moisture_level  \
6  Bok Choy  full sunlight or partial shade   well drained     consistent   

                  hardiness_zones lifecycle  difficulty       height  \
6  2, 3, 4, 5, 6, 7, 8, 9, 10, 11    annual           2  1 to 2 feet   

          spread time_to_mature flower_color foliage_color  \
6  1

Wrote a function to get top 10 recommended vegetables and then pull their information from our veg_info dataframe.

In [30]:
def pull_veggie(vegetable):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    Note: input needs to be a string from with a capitalized first letter
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg:
                    created_df = created_df.append(veg_info.loc[[num]])
                    
    return created_df

In [31]:
pull_veggie('Beets')

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers
6,Bok Choy,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10, 11",annual,2,1 to 2 feet,1 to 1.5 feet,45 days,NaN,medium green,"cushion, mound, or clump",low,Not native to North America - not known in the...,none,edible landscaping
8,Brussel Sprouts,full,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,2,2 to 3 feet,1.5 to 2 feet,26 - 31 weeks,NaN,medium green,upright,high,Not native to North America - not known in the...,Frost improves flavor. Harvest often continue...,none
20,Leeks,full sunlight or partial shade,well drained and high fertility,consistent,"7, 8",annual,2,2 to 3 feet,0.5 to 1 foot,120 - 150 days,NaN,medium and dark green,upright/fan-shaped,medium (more tolerant than other onions),Not native to North America - cultivated in Eg...,none,edible landscaping
23,Onions,full,well drained and high fertility,plentiful,"5, 6",annual,2,1 to 3 feet,0.5 to 1 foot,90 - 120 days,NaN,medium green,upright,medium,Not native to North America - unknown in wild....,none,edible landscaping
29,Radishes,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1,0.5 to 1.5 feet,0.5 to 0.75 feet,35 - 45 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - Mediterranean or...,none,none
32,Shallots,full sunlight or partial shade,well drained,consistent,"4, 5, 6, 7, 8, 9, 10",perennial,2,1 to 2 feet,0.5 to 1 foot,90 days,NaN,medium green,upright,medium,Not native to North America - Middle East origin,none,edible landscaping
36,Swiss Chard,full sunlight or partial shade,well drained,consistent,"6, 7, 8, 9, 10",annual,1,1 to 3 feet,0.5 to 2 feet,50 - 60 days,NaN,"medium/dark green, red, purple, and yellow","cushion, mound, or clump, and upright",medium,Not native to North America - domestivated fro...,none,edible landscaping
38,Turnips,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,1,1 to 1.5 feet,6 to 8 inches,40 - 55 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - not known in the...,none,none


Additional input, skill_level = user's gardeneing skill level, 

In [32]:
def pull_veggie_v2(vegetable, skill_level):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    Note: input needs to be a string from with a capitalized first letter
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg and veg_info['difficulty'][num] <= skill_level:
                    created_df = created_df.append(veg_info.loc[[num]])
                    
    return created_df

In [33]:
pull_veggie_v2('Beets', 1)

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers
29,Radishes,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1,0.5 to 1.5 feet,0.5 to 0.75 feet,35 - 45 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - Mediterranean or...,none,none
36,Swiss Chard,full sunlight or partial shade,well drained,consistent,"6, 7, 8, 9, 10",annual,1,1 to 3 feet,0.5 to 2 feet,50 - 60 days,NaN,"medium/dark green, red, purple, and yellow","cushion, mound, or clump, and upright",medium,Not native to North America - domestivated fro...,none,edible landscaping
38,Turnips,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,1,1 to 1.5 feet,6 to 8 inches,40 - 55 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - not known in the...,none,none


Additional input, hardiness_zone = user's hardiness zone.

NOTE: I kept the hardiness_zone data as a string so I could use `in` to see if the user's specified zone was in the data

In [34]:
def pull_veggie_v3(vegetable, skill_level, hardiness_zone):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    3) Check if the difficuly level is <= the user's specificed level
    4) Check if the user's hardiness zone is in the top 10 veggie choices
    Note: input needs to be a string from with a capitalized first letter 
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg and veg_info['difficulty'][num] <= skill_level and (hardiness_zone in veg_info['hardiness_zones'][num]):
                    created_df = created_df.append(veg_info.loc[[num]])
                    
    return created_df

In [35]:
pull_veggie_v3('Beets', 1, '4')

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers
29,Radishes,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1,0.5 to 1.5 feet,0.5 to 0.75 feet,35 - 45 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - Mediterranean or...,none,none
38,Turnips,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,1,1 to 1.5 feet,6 to 8 inches,40 - 55 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - not known in the...,none,none


In [36]:
veg_info.head(1)

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use
0,Artichokes,full sunlight,well drained,high,"6, 7, 8, 9",perennial,3,3 to 6 feet,2 to 4 feet,85 - 100 days,violet,light green,spreading mass/upright,high,Not native to North America,none,edible landscaping


In [37]:
#choose additional output data

def pull_veggie_v6(vegetable, skill_level, hardiness_zone):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    3) Check if the difficuly level is <= the user's specificed level
    4) Check if the user's hardiness zone is in the top 10 veggie choices
    5) Print some important facts about the plant
    Note: vegetable = string, skill_level = int, hardiness_zone = int as a string 
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg and veg_info['difficulty'][num] <= skill_level and (hardiness_zone in veg_info['hardiness_zones'][num]):
                    created_df = created_df.append(veg_info.loc[[num]])
    
    if created_df.shape[0] == 0:
        return "Unfortunately it doesn't look like we found any great matches for you, try adjusting your parameters"
    
    print("Our top picks for you are...")
    print()
    for num in range(0, created_df.shape[0]): 
        print(f"#{num+1} - {veg_lst[num]}!")
        print()
        print(f"A few important things to know before you grow {veg_lst[num]}:")
        print()
        print(f"1) Optimal sun exposure: {created_df.iloc[num][1]}")
        print()
        print(f"2) Ideal soil conditions: {created_df.iloc[num][2]}")
        print()
        print(f"3) Moisture level: {created_df.iloc[num][3]}")
        print()
        print(f"4) Expected spread: {created_df.iloc[num][7]}")
        print()
        print(f"5) Expected height: {created_df.iloc[num][8]}")
        print()
        print(f"6) Estimated time to harvest: {created_df.iloc[num][9]}")
        print()
        print("*" * 45)
        print()

In [38]:
pull_veggie_v6('Beets', 2, '4')

Our top picks for you are...

#1 - Radishes!

A few important things to know before you grow Radishes:

1) Optimal sun exposure: full sunlight or partial shade

2) Ideal soil conditions: low fertility

3) Moisture level: medium

4) Expected spread: 1 to 2 feet

5) Expected height: 0.5 to 1 foot

6) Estimated time to harvest: 4 - 7 weeks

*********************************************

#2 - Turnips!

A few important things to know before you grow Turnips:

1) Optimal sun exposure: full sunlight or partial shade

2) Ideal soil conditions: well drained

3) Moisture level: consistent

4) Expected spread: 1 to 2 feet

5) Expected height: 1 to 1.5 feet

6) Estimated time to harvest: 45 days

*********************************************

#3 - Rutabaga!

A few important things to know before you grow Rutabaga:

1) Optimal sun exposure: full

2) Ideal soil conditions: well drained 

3) Moisture level: consistent

4) Expected spread: 2 to 3 feet 

5) Expected height: 1.5 to 2 feet

6) Estimated 

In [39]:
veg_info.iloc[1][4]

'3, 4, 5, 6, 7, 8, 9, 10, 11'